In [1]:
pip install praw

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
print(praw.__version__)


7.8.1


In [4]:
import praw
import time
import json
from datetime import datetime, timezone


reddit = praw.Reddit(
    client_id='Q5pgjWDNQt3Aod_7TASUEA',
    client_secret='WMCAnsupIwLvX1mOD3gwOQPoYaTPcg',
    user_agent='windows:reddit.extractor:v1.0 (by /u/Plane_Picture_4411)'
)
reddit.read_only = True

subreddits = ['Notion', 'Artificial', 'SmallBusiness', 'Fitness']  
num_posts_per_subreddit = 1000

all_posts = []

for sub in subreddits:
    print(f"\n Scraping r/{sub}...")
    subreddit = reddit.subreddit(sub)
    count = 0

    try:
        for post in subreddit.top(limit=num_posts_per_subreddit):  
            time.sleep(2)  

            try:
                post.comments.replace_more(limit=0)
                top_comments = [comment.body for comment in post.comments[:10]]

                post_data = {
                    'title': post.title,
                    'score': post.score,
                    'url': post.url,
                    'created_utc': post.created_utc,
                    'created_time': datetime.fromtimestamp(post.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'author': post.author.name if post.author else '[deleted]',
                    'comments_count': post.num_comments,
                    'subreddit': sub,
                    'text': post.selftext,
                    'top_comments': top_comments
                }

                all_posts.append(post_data)
                count += 1
                print(f"r/{sub}: {count} posts scraped", end='\r')

            except Exception as inner_error:
                print(f"\n Error in post: {inner_error}")
                continue

    except Exception as outer_error:
        print(f"\n Failed to scrape r/{sub}: {outer_error}")
        continue
                                                  4
        
    print(f"\n Done with r/{sub}: {count} posts scraped.")

with open('reddit_posts_per_sub.json', 'w', encoding='utf-8') as f:
    json.dump(all_posts, f, ensure_ascii=False, indent=4)

print(f"\n Total posts scraped: {len(all_posts)}")
print(" Saved to reddit_posts_per_sub.json")



 Scraping r/Notion...
r/Notion: 999 posts scraped
 Done with r/Notion: 999 posts scraped.

 Scraping r/Artificial...
r/Artificial: 1000 posts scraped
 Done with r/Artificial: 1000 posts scraped.

 Scraping r/SmallBusiness...
r/SmallBusiness: 995 posts scraped
 Done with r/SmallBusiness: 995 posts scraped.

 Scraping r/Fitness...
r/Fitness: 994 posts scraped
 Done with r/Fitness: 994 posts scraped.

 Total posts scraped: 3988
 Saved to reddit_posts_per_sub.json


In [5]:
import pandas as pd

df = pd.DataFrame(all_posts)

df['top_comments'] = df['top_comments'].apply(lambda comments: ' ||| '.join(comments))

df.to_csv('reddit_posts_per_sub.csv', index=False, encoding='utf-8')

print(f"\n Data saved to reddit_posts_per_sub.csv")



 Data saved to reddit_posts_per_sub.csv


In [11]:
data=pd.read_csv("reddit_posts_per_sub.csv")
data

,title,score,url,created_utc,created_time,author,comments_count,subreddit,text,top_comments
0,"Notion, we need to talk: without an offline mo...",5548,https://www.reddit.com/r/Notion/comments/kqlfo...,1.609804e+09,2021-01-04 23:53:58,JoJokerer,226,Notion,"An open letter to Notion,\n\nI've evaluated so...",This is the angriest upvote I’ve ever given. |...
1,I'm just going to leave this here...,4627,https://i.redd.it/p7kakryw7y5e1.png,1.733805e+09,2024-12-10 04:27:32,carlpadonosk,32,Notion,NaN,Me scouring the internet for a custom icon to ...
2,ouch this one hurts :|,3719,https://i.imgur.com/K2MYOEf.png,1.606835e+09,2020-12-01 15:09:09,whoibrar,65,Notion,NaN,"While getting into Notion, I was also sorting ..."
3,So true,3707,https://i.redd.it/e29f98hl0qo61.png,1.616480e+09,2021-03-23 06:20:13,Cam223,60,Notion,NaN,"Weirdly enough, I hate using Notion as a note-..."
4,What do you mean this is not work,3584,https://i.imgur.com/4FUEZWH.jpg,1.623255e+09,2021-06-09 16:14:24,jerrygoyal,34,Notion,NaN,Damn! I created new notion templates for proce...
...,...,...,...,...,...,...,...,...,...,...
3983,Physique Phriday,200,https://www.reddit.com/r/Fitness/comments/1i3d...,1.737108e+09,2025-01-17 10:00:42,AutoModerator,169,Fitness,**Welcome to the Physique Phriday thread**\n\n...,Been cutting hard and lifting since May of las...
3984,Victory Sunday,190,https://www.reddit.com/r/Fitness/comments/1i4v...,1.737281e+09,2025-01-19 10:00:42,AutoModerator,183,Fitness,**Welcome to the Victory Sunday Thread**\n\nIt...,Currently at the gym and just caught a glimpse...
3985,Victory Sunday,179,https://www.reddit.com/r/Fitness/comments/1hzj...,1.736676e+09,2025-01-12 10:00:40,AutoModerator,164,Fitness,**Welcome to the Victory Sunday Thread**\n\nIt...,I can finally curl 30lbs and squat 120lbs. I k...
3986,Gym Story Saturday,160,https://www.reddit.com/r/Fitness/comments/1i9h...,1.737789e+09,2025-01-25 07:09:30,FGC_Valhalla,193,Fitness,Hi! Welcome to your weekly thread where you ca...,This week the front desk people didn't show up...


In [12]:
data=pd.read_csv("reddit_posts_per_sub.csv")
data.columns

Index(['title', 'score', 'url', 'created_utc', 'created_time', 'author',
       'comments_count', 'subreddit', 'text', 'top_comments'],
      dtype='object')

In [13]:
data["subreddit"].value_counts()

subreddit
Artificial       1000
Notion            999
SmallBusiness     995
Fitness           994
Name: count, dtype: int64